<a href="https://colab.research.google.com/github/vlad-uve/CAE-MNIST/blob/main/notebooks/CAE_base_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries & Constants

In [ ]:
# PyTorch core
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Data loading and transformations
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Model summary utility
from torchsummary import summary

# Learning rate scheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Plotting
import matplotlib.pyplot as plt

In [ ]:
# set device to GPU if available, else use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# set random seed for reproducibility
torch.manual_seed(0)

# Define Base Model

In [ ]:
# define baseline model architecture
base_n_channels = [32, 32, 64]
base_latent_dim = 32

# assign dataloaders
base_train_dataloader = train_dataloader
base_validation_dataloader = validation_dataloader

# initialize baseline model
base_model = AutoEncoder(
    n_channels=base_n_channels,
    latent_dim=base_latent_dim
).to(device)

# define optimizer for baseline model
base_optimizer = optim.Adam(base_model.parameters(), lr=1e-3)

# define learning rate scheduler (monitors validation loss plateau)
base_scheduler = ReduceLROnPlateau(
    base_optimizer,
    mode='min',
    threshold=1e-3,
    patience=3,
    factor=0.5
)

# define baseline number of training epochs
base_num_epoch = 20

# print model summary for baseline model
summary(base_model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 14, 14]             544
          Identity-2           [-1, 32, 14, 14]               0
            Conv2d-3             [-1, 32, 7, 7]          16,416
          Identity-4             [-1, 32, 7, 7]               0
            Conv2d-5             [-1, 64, 4, 4]          18,496
          Identity-6             [-1, 64, 4, 4]               0
           Flatten-7                 [-1, 1024]               0
            Linear-8                   [-1, 32]          32,800
           Encoder-9                   [-1, 32]               0
           Linear-10                 [-1, 1024]          33,792
        Unflatten-11             [-1, 64, 4, 4]               0
  ConvTranspose2d-12             [-1, 32, 7, 7]          18,464
         Identity-13             [-1, 32, 7, 7]               0
  ConvTranspose2d-14           [-1, 32,

# Train Base Model

In [ ]:
# train baseline model
base_model, base_loss = run_model_training(
    model=base_model,
    train_dataloader=base_train_dataloader,
    validation_dataloader=base_validation_dataloader,
    optimizer=base_optimizer,
    scheduler=base_scheduler,
    num_epoch=base_num_epoch
)


TRAINING IS STARTED:


## Export and Import

In [ ]:
# save baseline model weights and loss history locally
torch.save(base_model.state_dict(), 'base_model.pth')
torch.save(base_loss, 'base_loss.pth')

# from google.colab import files
# files.download('base_model.pth')
# files.download('base_loss.pth')

In [ ]:
# Option 1: manual upload (Colab only)
# from google.colab import files
# uploaded = files.upload()

# Option 2: load from file path (recommended for reproducibility)
base_model = AutoEncoder(n_channels=base_n_channels, latent_dim=base_latent_dim).to(device)
base_model.load_state_dict(torch.load('base_model.pth', map_location=torch.device(device)))
base_model.eval()

base_loss = torch.load('base_loss.pth')